# Task 2 - Landsat 8 false color image

## General instructions

First, update the following cell to have a link to your Homework 4 GitHub repository

https://github.com/saingersoll/eds220-hwk-4

Add comments for all your code and commit as needed. Err on the side of commenting and commiting too much for now.

## Overview
The goal of this task is to create a false color image showing the fire scar of the Thomas fire in 2017. 
Read more about false color images here:
[NASA Earth Observatory - Why is that Forest Red and that Cloud Blue?
How to Interpret a False-Color Satellite Image](https://earthobservatory.nasa.gov/features/FalseColor)

## About the data

In this task you will use two datsets:

**First dataset**

A simplified collection of bands (red, green, blue, near-infrared and shortwave infrared) from the Landsat Collection 2 Level-2 atmosperically corrected surface reflectance data, collected by the Landsat 8 satellite. 

Information about Landsat bands from USGS:

- [What are the band designations for the Landsat satellites?](https://www.usgs.gov/faqs/what-are-band-designations-landsat-satellites)

-[Common Landsat Band Combinations](https://www.usgs.gov/media/images/common-landsat-band-combinations)

-[How do I use a scale factor with Landsat Level-2 science products?](https://www.usgs.gov/faqs/how-do-i-use-a-scale-factor-landsat-level-2-science-products)


The data was accessed and pre-processed in the Microsoft Planetary Computer to remove data outside land and coarsen the spatial resolution ([Landsat Collection in MPC](https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2)). Data should be used for visualization purposes only. 


**Second dataset**

A shapefile of fire perimeters in California during 2017. 
The [complete file can be accessed in the CA state geoportal](https://gis.data.ca.gov/datasets/CALFIRE-Forestry::california-fire-perimeters-all-1/about).

## File management
All your data files must be  inside a directory named 'data' **in your gitignore**. 

## Data access
All data is in the Canvas-week7-homework zip file.

## FINAL CODE

You will use the next cell at the end of the task. Leave it blank for now.

In [1]:
### ===== FINAL CODE =====

## Import libraries and functions

In [2]:
# reading in libraries
import os
import numpy as np
import pandas as pd

import geopandas as gpd
import xarray as xr
import rioxarray as rioxr
import rasterio
from shapely.geometry import Polygon

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

## Import data

In [3]:
# reading in the data
landsat = rioxr.open_rasterio(os.path.join(os.getcwd(), 'data','landsat8-2018-01-26-sb-simplified.nc'))         # netcdf file

# confirming the dataset loaded properly
landsat

<xarray.Dataset>
Dimensions:      (y: 731, x: 870, band: 1)
Coordinates:
  * y            (y) float64 3.952e+06 3.952e+06 ... 3.756e+06 3.755e+06
  * x            (x) float64 1.213e+05 1.216e+05 ... 3.557e+05 3.559e+05
  * band         (band) int64 1
    spatial_ref  int64 0
Data variables:
    red          (band, y, x) float64 ...
    green        (band, y, x) float64 ...
    blue         (band, y, x) float64 ...
    nir08        (band, y, x) float64 ...
    swir22       (band, y, x) float64 ...

In [8]:
# reading in the data
ca = gpd.read_file('data/California_Fire_Perimeters/California_Fire_Perimeters_2017.shp')

# confirming the datasets loaded properly
ca.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   index       608 non-null    int64   
 1   OBJECTID    608 non-null    int64   
 2   YEAR_       608 non-null    object  
 3   STATE       608 non-null    object  
 4   AGENCY      608 non-null    object  
 5   UNIT_ID     608 non-null    object  
 6   FIRE_NAME   597 non-null    object  
 7   INC_NUM     608 non-null    object  
 8   ALARM_DATE  608 non-null    object  
 9   CONT_DATE   606 non-null    object  
 10  CAUSE       608 non-null    float64 
 11  C_METHOD    608 non-null    float64 
 12  OBJECTIVE   608 non-null    float64 
 13  GIS_ACRES   608 non-null    float64 
 14  COMMENTS    148 non-null    object  
 15  COMPLEX_NA  39 non-null     object  
 16  COMPLEX_IN  39 non-null     object  
 17  IRWINID     7 non-null      object  
 18  FIRE_NUM    40 non-null     object  
 19  

## Exploration 

Use the next cell(s) to perform relevant explorations of these datasets.

## Cleaning the `landsat` dimensions

In [5]:
# removing band dimension from landsat
# original dimensions and coordinates
#print(landsat.dims, landsat.coords,'\n')

# remove length 1 dimension (band)
#landsat = landsat.squeeze()
#print(landsat.dims, landsat.coords,'\n')

# remove coordinates associated to band
#landsat = landsat.drop('band')
#print(landsat.dims, landsat.coords)

In [6]:
landsat

<xarray.Dataset>
Dimensions:      (y: 731, x: 870, band: 1)
Coordinates:
  * y            (y) float64 3.952e+06 3.952e+06 ... 3.756e+06 3.755e+06
  * x            (x) float64 1.213e+05 1.216e+05 ... 3.557e+05 3.559e+05
  * band         (band) int64 1
    spatial_ref  int64 0
Data variables:
    red          (band, y, x) float64 ...
    green        (band, y, x) float64 ...
    blue         (band, y, x) float64 ...
    nir08        (band, y, x) float64 ...
    swir22       (band, y, x) float64 ...

### `landsat` Geospatial Attributes

#### Unexpected Error, why is it not reading y,x as the data array shows above?

In [7]:
# check geospatial attributes
print('# bands: ', landsat.rio.count)
print('height: ', landsat.rio.height)
print('width: ', landsat.rio.width, '\n')

print('spatial bounding box: ')
print(landsat.rio.bounds(), '\n')

print('CRS: ', landsat.rio.crs)

InvalidDimensionOrder: Invalid dimension order. Expected order: ('band', 'y', 'x'). You can use `DataArray.transpose('band', 'y', 'x')` to reorder your dimensions.

### Visualizing `landsat` Data

#### Not plotting, similar to class issue. Code matches Carmen's notebooks (rioxarray, start manipulation)

In [ ]:
size = 4
aspect = landsat.rio.width/landsat.rio.height
landsat.plot.imshow(size = size, aspect = aspect)

### CA Fire Perimeter Data Visualization 

In [ ]:
# confirming the dataset loaded properly by plot
ca.plot()

## Updates

If there are any updates needed to the data to plot the true and false color images, perform them in the next cells (you may add as many as you need). 
Comment appropriately.

In [ ]:
# your code here

## True color image

Without creating any new variables:

Select the red, green and blue variables (in that order) of the `xarray.Dataset`, convert it to a `numpy.array` using the `to_array()` method, and then use `.plot.imshow()` to create an RGB image with the data. (There will be a warning)

In [ ]:
# your code here

Adjust the scale used for plotting the bands to get a true color image. 
HINT: [Check the `robust` parameter](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html)
The issue here is the clouds: their RGB values are outliers and cause the other values to be squished when plotting. 

In [ ]:
# your code here

## False color image
Without creating any new variables:

Create a false color image by plotting the short-wave infrared (swir22), near-infrared, and red variables (in that order).

In [ ]:
# your code here

## Updates

If there are any updates needed to the data to create the map, perform them in the next cells (you may add as many as you need). 
Comment appropriately.

In [ ]:
# your code here

## Map

Create a map showing the shortwave infrared/nir/ ed false color image together with the Thomas fire perimeter. 
Customize it appropriately.

In [ ]:
# your code here

## Final code

Collect all the relevant code into the first blank cell of the notebook titled "FINAL CODE". This single cell will have the end-to-end workflow: from importing libraries and loading the data, to producing the graph. The only ouput of this cell should be the plot you produced in the previous exercise. 

Add divisions for relevant sections and comment appropriately.